# How to run wastebot

This notebook is to walk through how to run the wastebot. 

## Import modules
A module is a set of functions that can do lots of useful stuff. Kevin and Jonathan made a lot of convenient functions and packed them as modules, which are 
 - robot module 
 - procedures module
 
but `procedures` module call `robot` module internally, so you only need to import this module (more specifically, `DummyExperiment` function)

In a nutshell, you call this section everytime you run the robot 

The next cell also contains all the python module required to analyse the data, such as:
- Communication with R
- Creation of boxplot
- Creation of 3D plots
- Creation of widgets (selection of parameter and dynamic plots)

In [1]:
import os
import sys
import inspect
import time
import csv
import numpy as np
import datetime
import subprocess 

# OpenCV for image acquistion from webcan and subsequent analysis
import cv2

# load Experiment1 procedures module, which will allow access to all general
# procedure functions and those from Experiment1
#from procedures.procedures import Experiment1
from procedures.procedures import DummyExperiment
# add root path to access project modules
HERE_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(HERE_PATH)

#allowed communication with R
from scipy.integrate import odeint
from matplotlib import pyplot as plt

#import for boxplot
import string
import plotly.plotly as py
import plotly.graph_objs as go

#from mpl_toolkits.mplot3d import Axes3D # for 3d plot
% matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()
sns.set_context("poster")
sns.set_style("whitegrid")

## dynamic plot
from ipywidgets import widgets, Button, Layout
from IPython.display import display

## import packages to communicate with R
from numpy import *
import scipy as sp
from pandas import *
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
#import pandas.rpy.common as com

%load_ext rpy2.ipython
from rpy2.robjects import r, pandas2ri
from rpy2 import robjects
from rpy2.robjects.packages import importr

## import ggplot (python package)
#from ggplot import *

# import R packages

d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}

from rpy2.robjects.packages import importr
base = importr('base')
utils = importr('utils')
gc = importr('growthcurver')
plyr = importr('plyr')
ggplot2 = importr('ggplot2')
reshape2 = importr('reshape2')
dat_tab = importr('data.table')
stats = importr('stats', robject_translations = d)
from rpy2.robjects import pandas2ri
pandas2ri.activate()

## Initialising the robot
Robot needs a configuration file to know what's available or not (e.g. Z-axis is available? A pipette is equipped?, etc).  

In [2]:
# create general procedures object, no need to link configfiles
# this initializes all parts of the robot
exp1 = DummyExperiment(robot_configfile="robot/robot_config.json")

# define initial position for X,Y,Z
init_x = 0
init_y = 0
init_z = 50
# pitch between wells cf. http://ibidi.com/xtproducts/en/ibidi-Labware/m-Plates/m-Plate-24-Well
well_pitch = 19.30 

INFO:commanduino.CommandManager:Found CommandManager on port "COM14", init time was 1.045 seconds
INFO:commanduino.CommandManager:Device "Y" with id "Y" and of type "LINEARACCELSTEPPER" found in 0.016s
INFO:commanduino.CommandManager:Device "Y" with id "Y" and of type "LINEARACCELSTEPPER" found in the register, creating it
INFO:commanduino.CommandManager:Device "X" with id "X" and of type "LINEARACCELSTEPPER" found in 0.016s
INFO:commanduino.CommandManager:Device "X" with id "X" and of type "LINEARACCELSTEPPER" found in the register, creating it
INFO:commanduino.CommandManager:Device "Z" with id "Z" and of type "LINEARACCELSTEPPER" found in 0.016s
INFO:commanduino.CommandManager:Device "Z" with id "Z" and of type "LINEARACCELSTEPPER" found in the register, creating it


## Moving robot
There are a number of functions defined in `robot.py`, but what we need is essentially only one: `move_to()` 

For more functions, see `robot.py` in `C:\Users\turbidostat\Desktop\wastebot\wastebot_complete\software\python\robot`

In [ ]:
# move the robot to the initial position defined above 
exp1.robot.move_to([init_x, init_y, init_z])

In [ ]:
exp1.robot.p.aspirate(25)

In [ ]:
# move the robot to find the initial position 
exp1.robot.move_to([0, 200, init_z])

In [ ]:
# move the robot to the first well
exp1.robot.move_to([0, 0, init_z])

In [7]:
# move the robot to the last well
exp1.robot.move_to([19.3*2.5, 19.3*0.5, init_z])

In [ ]:
# test the plate shaking: move the robot to (x,y,z)=(100,120,20)
exp1.robot.move_to_firstxy([19.3*2.5+1.8, 19.3*4-1.8, 98])
distance = 1.5
for _ in range(10):
    exp1.robot.move_to([19.3*2.5+1.8-distance, 19.3*4-1.8, 98])
    exp1.robot.move_to([19.3*2.5+1.8+distance, 19.3*4-1.8, 98])
    exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8, 98])
    exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8+distance, 98])
    exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8-distance, 98])
    exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8, 98])
exp1.robot.move_to_firstz([19.3*2.5+1.8, 19.3*4-1.8, init_z])

In [ ]:
# move the robot to the center of the plate
exp1.robot.move_to_firstxy([19.3*2.5+1.8, 19.3*4-1.8, 85])

In [ ]:
# you can also move each axis individually, or only xy together 
## move camera to the last well
for _ in range(3):
    exp1.robot.xy.move_to([19.3*5,19.3*3])

In [ ]:
exp1.robot.home()

Initialising webcam 
A webcam (Microsoft Lifecam) also needs to be initialised. What we do here are:
 - Connect to a webcam
 - Set the image resolution for captured image 
 - Define a useful function to retrieve one frame from the webcam

In [30]:
# connect to a webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280) # set the resolution
cap.set(4, 1024)  # Width of the frames in the video stream.
cap.set(16, 0.1) # set exporesure time 
cap.set(11, 45) # set brightness [0:100]
cap.set(12, 65) # set contrast [0:100]
cap.set(13, 0) # set saturation [-100:100]
cap.set(14, 360) # set hue 
cap.set(15, 1) # set gain
cap.set(18, 0) # set white balance

# see https://stackoverflow.com/questions/11420748/setting-camera-parameters-in-opencv-python for more options 
#0. CV_CAP_PROP_POS_MSEC Current position of the video file in milliseconds.
#1. CV_CAP_PROP_POS_FRAMES 0-based index of the frame to be decoded/captured next.
#3. CV_CAP_PROP_POS_AVI_RATIO Relative position of the video file
#4. CV_CAP_PROP_FRAME_WIDTH Width of the frames in the video stream.
#5. CV_CAP_PROP_FRAME_HEIGHT Height of the frames in the video stream.
#6. CV_CAP_PROP_FPS Frame rate.
#7. CV_CAP_PROP_FOURCC 4-character code of codec.
#8. CV_CAP_PROP_FRAME_COUNT Number of frames in the video file.
#9. CV_CAP_PROP_FORMAT Format of the Mat objects returned by retrieve() .
#10. CV_CAP_PROP_MODE Backend-specific value indicating the current capture mode.
#11. CV_CAP_PROP_BRIGHTNESS Brightness of the image (only for cameras).
#12. CV_CAP_PROP_CONTRAST Contrast of the image (only for cameras).
#13. CV_CAP_PROP_SATURATION Saturation of the image (only for cameras).
#14. CV_CAP_PROP_HUE Hue of the image (only for cameras).
#15. CV_CAP_PROP_GAIN Gain of the image (only for cameras).
#16. CV_CAP_PROP_EXPOSURE Exposure (only for cameras).
#17. CV_CAP_PROP_CONVERT_RGB Boolean flags indicating whether images should be converted to RGB.
#18. CV_CAP_PROP_WHITE_BALANCE Currently unsupported
#19. CV_CAP_PROP_RECTIFICATION Rectification flag for stereo cameras (note: only supported by DC1394 v 2.x backend currently)


# Captures a single image from the camera and returns it in PIL format
def get_image():
    # read is the easiest way to get a full image out of a VideoCapture object.
    retval, im = cap.read()
    return im

## Saving images 
Next step is to capture an image from the webcam and save it as an image file (e.g. PNG file). 

In [31]:
# We save image 30 times but throw them away --- This gives time for the webcam to auto-focus 
for _ in xrange(30):
    temp = get_image()

# get a test image first to check x,y position 
timenow = datetime.now().replace(microsecond=0)
# Set a file name to be image     
testimgfile = "test-%s-%s.png" % (timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))

# imwrite() is the function to save the captured image 
cv2.imwrite(testimgfile, get_image())
print 'saving an image as %s' % testimgfile
time.sleep(1) # give some time for a PC to save the image before doing anything (just in case ;)

saving an image as test-20180706-101927.png


In [29]:
# set directory name 
directory = 'webcamtest-%s' % (timenow.strftime("%Y%m%d"))
# create a directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)
    
# We save image 30 times but throw them away --- This gives time for the webcam to auto-focus 
for _ in xrange(30):
    temp = get_image()

# get a test image first to check x,y position 
timenow = datetime.now().replace(microsecond=0)
# Set a file name to be image     
testimgfile = "./%s/test-%s-%s.png" % (directory, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))

# imwrite() is the function to save the captured image 
cv2.imwrite(testimgfile, get_image())
print 'saving an image as %s' % testimgfile
time.sleep(1) # give some time for a PC to save the image before doing anything (just in case ;)

saving an image as ./webcamtest-20180706/test-20180706-084953.png


In [17]:
# set directory name 
directory = 'Biolum_test_strains_expe1-%s' % (timenow.strftime("%Y%m%d"))
# create a directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)
    

for k in range(0, 3000):
    
    # get current time
    timenow = datetime.now().replace(microsecond=0)
    # Ramp the camera - these frames will be discarded and are only used to allow v4l2
    # to adjust light levels, if necessary
    for foo in xrange(30):
        temp = get_image()
    testimgfile = "./%s/test-%s-%s.png" % (directory, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))

    # imwrite() is the function to save the captured image 
    cv2.imwrite(testimgfile, get_image())
    print 'saving an image as %s' % testimgfile
    
    time.sleep(7*60) # wait 2 min
    

saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-172733.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-173504.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-174235.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-175006.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-175737.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-180508.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-181239.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-182010.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-182741.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-183512.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-184243.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180705-185014.png
saving an image as ./Biolum_

saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-061415.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-062146.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-062917.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-063648.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-064419.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-065150.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-065921.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-070652.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-071423.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-072154.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-072925.png
saving an image as ./Biolum_test_strains_expe1-20180705/test-20180706-073656.png
saving an image as ./Biolum_

KeyboardInterrupt: 

Now you should have a PNG image saved in the same directory as this Notebook file. Open the file and check if the program saved an image correctly as expected ---- i.e. the image is from Lifecam, not from a built-in webcam on the laptop. Also check if the image is not blurred to check if autofocus works fine. 

## Analysing an image 
After you capture an image, you need to analyse the image. A captured image is basically a 3-layered matrix, each of which corresponds to Red, Green, and Blue image (sometimes you have another layer for alpha, transparency). 

Each layer is a matrix, so you can do any matrix opetion for each layer. For example, you can crop a small area from the whole image and save as another file.

In this program, a mask in form of circle is used to fit with the shape of a 24 well plate. The size of the circle can be change by increasing or decreasing the "cropsize" value. The centre of the circle can be moved by changing the values of "cent_x"and "cent_y".

In [ ]:
## Crop elliptical region from image
# Setting initial values 
cent_x = 628
cent_y = 432
cropsize = 60
# capture a frame 
img = get_image()
# save the frame as an image 
imgfile = "./wastebot-%s-%s.png" % (timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
#cv2.imwrite(imgfile, img)
# analyse image real time
crop_img_grey = cv2.cvtColor(img[cent_y-cropsize:cent_y+cropsize, cent_x-cropsize:cent_x+cropsize], cv2.COLOR_BGR2GRAY)

# do the circle
height = cropsize
width = cropsize
height, width = crop_img_grey.shape
circle_img = np.zeros((height, width), np.uint8)
cv2.circle(circle_img, (height/2, width/2), 1, 1, thickness = 60)
# paste the circle mask to the crop image
masked_data = cv2.bitwise_and (crop_img_grey, crop_img_grey, mask = circle_img)

outfile = imgfile.replace("wastebot", "cropped_wastebot2")
outfilegrey = imgfile.replace(".png", "-grey.png")

cv2.imwrite(outfilegrey, masked_data)

# Define the folders where datas will be stored

Datas of the experiment are stored into 3 different folders:
- "directory_expe" contains all the data
- "directory_fullpic"contains the full picture taken during the analysis
- "directory_croppic" contains the picture with after the circle crop treatment

In [ ]:
timenow = datetime.now().replace(microsecond=0)   # Get the actual time

# Create folder to store all the data for the experiment
directory_expe = 'TRP-YE_growth-%s-%s' % (timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
# create a directory if it doesn't exist
if not os.path.exists(directory_expe):
    os.makedirs(directory_expe)

# Create folder to store full pictures
directory_fullpic = './%s/full_picture-%s-%s' % (directory_expe, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
# create a directory if it doesn't exist
if not os.path.exists(directory_fullpic):
    os.makedirs(directory_fullpic)
    
# Create folder to store cropped pictures
directory_croppic = './%s/cropped_picture-%s-%s' % (directory_expe, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
# create a directory if it doesn't exist
if not os.path.exists(directory_croppic):
    os.makedirs(directory_croppic)


# change the working directory
#os.chdir(directory_expe)

# Define the different condition to prepare the different wells

### 1) Define the parameters of the experiment and save them into a .txt file

In [ ]:
## Define the experimental conditions/parametersc

style = {'description_width': 'initial'} ## Style  to see the full description

Cmin = widgets.BoundedFloatText(    ##Widget to define the minimal concentration tested
    min=0,
    max=1000,
    step=0.1,
    description='C.min (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(Cmin)   ## display the widget

Cmax = widgets.BoundedFloatText(    ##Widget to define the maximal concentration tested
    min=0,
    max=1000,
    step=0.1,
    description='C.max (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(Cmax)   ## display the widget

nb_cond = widgets.BoundedIntText(    ##Widget to define the number of condition wanted to test
    min=0,
    max=1000,
    step=1,
    description='Nb of tested condition:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(nb_cond)   ## display the widget


nb_cond_per_run = widgets.BoundedIntText(    ##Widget to define the number of condition wanted to test
    min=0,
    max=1000,
    step=1,
    description='Nb of different condition to test per run (including the blank):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(nb_cond_per_run)   ## display the widget

    
nb_replicate = widgets.BoundedIntText(    ##Widget to define the number of replicate per condition
    min=0,
    max=1000,
    step=1,
    description='Nb of replicate per condition:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(nb_replicate)   ## display the widget
    

Well_vol = widgets.BoundedFloatText(    ##Widget to define the value the volume in wells
    min=0,
    max=2.0,
    step=0.1,
    description='Volume in wells (ml):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(Well_vol)   ## display the widget

TRP_i_conc = widgets.BoundedFloatText(    ##Widget to define the value of the mother TRP concentration
    min=0,
    max=1001.00,
    step=0.01,
    description='TRP initial concentration (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(TRP_i_conc)   ## display the widget

YE_i_conc = widgets.BoundedFloatText(     ##Widget to define the value of the mother Yeast Extract concentration
    min=0,
    max=1001.00,
    step=0.01,
    description='Yeast extract initial concentration (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(YE_i_conc)   ## display the widget

NaCl_i_conc = widgets.BoundedFloatText(     ##Widget to define the value of the mother NaCl concentration
    min=0,
    max=1001.00,
    step=0.01,
    description='NaCl initial concentration (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(NaCl_i_conc)   ## display the widget

NaCl_f_conc = widgets.BoundedFloatText(   ##Widget to define the value of the final NaCl concentration (in wells)
    min=0,
    max=1001.00,
    step=0.01,
    description='NaCl final concentration (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(NaCl_f_conc)   ## display the widget

dil_inoc = widgets.BoundedFloatText(   ##Widget to define the value of the inoculum dilution
    min=0,
    max=1001.00,
    step=0.01,
    description='Dilution of the inoculum:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(dil_inoc)    ## display the widget


blank_val = widgets.BoundedFloatText(    ##Widget to define the concentration of the blank
    min=0,
    max=1001.00,
    step=0.01,
    description='Concentration for the blank (g/l):\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(blank_val)   ## display the widget


plate_row = widgets.BoundedIntText(    ##Widget to define the size of the plate: number of row
    min=0,
    max=1001,
    step=1,
    description='Number of row in the plate:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(plate_row)   ## display the widget


plate_col = widgets.BoundedIntText(    ##Widget to define the size of the plate: number of column
    min=0,
    max=1001,
    step=1,
    description='Number of column in the plate:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(plate_col)   ## display the widget


file_name = widgets.Text(            ##Widget to define the the name to give to the parameter file
    value='File name',
    placeholder='Type the name of the parameter file',
    description='String:\n',
    disabled=False, style = style,
    layout=Layout(width='75%', height='30px')
)

display(file_name)   ## display the widget


## Create button to display the table of preparation
def on_button_table_prep(b):
    conf_file = "./%s/%s-%s-%s.txt" % (directory_expe, file_name.value, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
    #names = "./%s/names.csv" % (directory_expe)
    print ('Detailled data will be written in %s' % conf_file)

    with open(conf_file, 'w') as conf_file:
        conf_file.write('C_min ='+ str(Cmin.value)+ '\n')
        conf_file.write('C_min ='+ str(Cmin.value)+'\n')
        conf_file.write('C_max ='+ str(Cmax.value)+'\n')
        conf_file.write('nb_of_cond ='+ str(nb_cond.value)+'\n')
        conf_file.write('nb_of_cond_per_run ='+ str(nb_cond_per_run.value)+'\n')
        conf_file.write('nb_of_replicate ='+ str(nb_replicate.value)+'\n')
        conf_file.write('vol_in_well ='+ str(Well_vol.value)+'\n')
        conf_file.write('TRP_conc_i ='+ str(TRP_i_conc.value)+'\n')
        conf_file.write('YE_conc_i ='+ str(YE_i_conc.value)+'\n')
        conf_file.write('NaCl_conc_i ='+ str(NaCl_i_conc.value)+'\n')
        conf_file.write('NaCl_conc_f ='+ str(NaCl_f_conc.value)+'\n')
        conf_file.write('dil_of_inoc ='+ str(dil_inoc.value)+'\n')
        conf_file.write('val_of_blank ='+ str(blank_val.value)+'\n')
        conf_file.write('nb_row_plate ='+ str(plate_row.value)+'\n')
        conf_file.write('nb_col_plate ='+ str(plate_col.value)+'\n')
        conf_file.write
   
button = widgets.Button(description="Make table of preparation")
display(button)

button.on_click(on_button_table_prep)

### 2) From the previous experimental condition file, calculate and prepare the experiment
Table of preparation per run and for all the conditions are saved into a .csv file

In [ ]:
# Read the configuration file
filename = ("./%s/%s-%s-%s.txt" % (directory_expe, file_name.value, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S")))
exec(open(filename).read())        # Execute the configuration file


# Define the concentration to test: list called "conc"
conc = [0] * nb_of_cond                           # Create an empty list
interval = (C_max-C_min)/(nb_of_cond-1)           # Calculate the interval between 2 concentrations
for i in range (0,(nb_of_cond-1)):                # Loop to fill the concentration list (without the last value)
    conc [i] = C_min + interval * i
conc[-1] = C_max                                  # Add the last value

print ("Concentrations to be prepared (in g/l):\n", conc, "\n")


# Create the list for the wells name: list called "wells"
alpha = list(string.ascii_uppercase)                 # Create a list with the alpahbet letters
wells= [0]*(nb_row_plate*nb_col_plate)               # Define the max number of wells per plate (depending the size of the plate)
rname = alpha [0:nb_row_plate]                       # Select the letters required according the number of row in the plate
cname = list(map(str, [x + 1 for x in list(range(nb_col_plate))]))      # Select the numbers required according the number of column in the plate

## For loop to create the list of wells (A1, A2, A3...)
for r in range(0,len(rname)) :         # Number of row (A, B, C, D)
    for c in range(0,len(cname)) :     # Number of column (1, 2, 3, 4, 5, 6)
        wells[c+(r*nb_col_plate)]= rname[r] + cname[c]


# Reciepe for the preparation of each condition
trp_conc = list(conc)                                            ## save the list under a new name
YE_conc = [x / 2 for x in trp_conc]                              ## define a list for YE concentration
trp2_vol = [(x * vol_in_well)/TRP_conc_i for x in trp_conc]      ## calculate the volume of TRP for each concentration
YE2_vol = [(x * vol_in_well)/YE_conc_i for x in YE_conc]         ## calculate the volume of YE for each concentration
nacl_vol = (NaCl_conc_f * vol_in_well)/NaCl_conc_i               ## calculate the volume of NaCl (same for each concentration)
nacl2_vol = [nacl_vol] * len(trp2_vol)                           ## create a list of NaCl volume for each concentration
inoc_vol = vol_in_well / dil_of_inoc                             ## calculate the volume of inoculum (same for each concentration)
inoc2_vol = [inoc_vol] * len(trp2_vol)                           ## Create a list of inoculum volume for each concentration
H2O_vol = [vol_in_well - (x + y + nacl_vol + inoc_vol) for x, y in zip(trp2_vol, YE2_vol)]  ## Calculate the volume of H2O needed
prep = [('test conc.', conc),                  ## Create a list composed of the previous lists
        ('volume TRP (ml)', trp2_vol),
        ('volume Y-E (ml)', YE2_vol),
        ('volume NaCl (ml)', nacl2_vol),
        ('volume inoculum (ml)', inoc2_vol),
        ('volume H2O (ml)', H2O_vol),
       ]
df_prep_all_conc = pd.DataFrame.from_items(prep)           ## Make a dataframe of the previous lists

# Save the table of preparation in a csv file: ".to_csv" function
prep_tot_file = "./%s/all_conditions-%s-%s.csv" % (directory_expe, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
df_prep_all_conc.to_csv(prep_tot_file, mode='a')

print ('Detailled data will be written in %s' % prep_tot_file,  "\n")

print(df_prep_all_conc, "\n")


# define the number of run needed
nb_run = math.ceil(nb_of_cond/(nb_of_cond_per_run - 1))
print('\nThe experiment will require ', nb_run, ' runs.\n\n')


#Shuffle the list "conc" to randomly select the runs
random.shuffle(conc)
print ("Random Shuffle of concentrations:\n", conc, "\n")


# Create empty dictionnaries required for the FOR loop
run = 'Run'      ## Name to give to each list in dictionnaries
Well = {}        ## Dictionary for the name of wells
conc_test_blk = {}
conc_test = {}   ## Dictionary for the TRP concentration tested
TRP_vol = {}     ## Dictionary for the volume of TRP
YE_conc = {}     ## Dictionary for the YE concentration tested
YE_vol={}        ## Dictionary for the volume of YE
nacl_vol = {}    ## Dictionary for the volume of NaCl
inoc_vol = {}    ## Dictionary for the volume of inoculum
H2O_vol = {}     ## Dictionary for the volume of H2O

# Loop to fill the previous dictionaries
for i in range (0,int(nb_run)):
    for c in range (0,len(conc)):
        conc_test_blk[run+str(i+1)] = [(conc[0+(i*(int(nb_of_cond_per_run)-1)):(int(nb_of_cond_per_run)-1)+(i*(int(nb_of_cond_per_run)-1))] + ['blank'])*nb_of_replicate]          # calculate the concentration for each run
        random.shuffle(conc_test_blk[run+str(i+1)][0])                                                       # shuffle randomly each list (per run)
        conc_test[run+str(i+1)] = [x if x!= 'blank'else val_of_blank for x in (conc_test_blk[run+str(i+1)][0])]
        TRP_vol[run+str(i+1)] = [(x * vol_in_well)/TRP_conc_i for x in (conc_test[run+str(i+1)])]  # calculate the volume of TRP in each well (per run)
        YE_conc[run+str(i+1)] = [x / 2 for x in (conc_test[run+str(i+1)])]                         # calculate the concentration of YE in each well (per run)
        YE_vol[run+str(i+1)] = [(x * vol_in_well)/YE_conc_i for x in YE_conc[run+str(i+1)]]           # calculate the volume of YE in each well (per run)
        nacl_v = (NaCl_conc_f * vol_in_well)/NaCl_conc_i                                              # calculate the volume of NaCl (same for each concentration)
        nacl_vol[run+str(i+1)] = list([nacl_v] * len(TRP_vol[run+str(i+1)]))                          # Create a list of NaCl volume for each TRP concentration (per run)
        inoc_v = vol_in_well / dil_of_inoc                                                            # calculate the volume of inoculum (same for each concentration)
        inoc_vol[run+str(i+1)] = list([inoc_v] * len(TRP_vol[run+str(i+1)]))                          # Create a list of inoculum volume for each TRP concentration (per run)
        H2O_vol[run+str(i+1)] = [vol_in_well - (x + y + nacl_v + inoc_v) for x, y in zip(TRP_vol[run+str(i+1)], YE_vol[run+str(i+1)])]   # Calculate the volume of H2O needed (per well, per run)

        Well[run+str(i+1)] = [0] * (len(TRP_vol[run+str(i+1)]))         # Create a list with the name of wells (per run)
        for w in range(0,len(TRP_vol[run+str(i+1)])):
            Well[run+str(i+1)][0+w] = wells[w]

all_dict = [Well,conc_test_blk,conc_test,TRP_vol,YE_conc, YE_vol, nacl_vol,inoc_vol,H2O_vol]       # Make a list containing all dictionnaries
# Create a dataframe from the lrevious list
prep_each_run = pd.DataFrame(all_dict, index =['Well','Place of blanks','Tested conc.','Volume TRP','tested YE conc.',
                                               'Volume YE', 'Volume NaCl','Volume inoculum','Volume H2O'])

# Save the dataframe into a CSV file
prep_file = "./%s/prep_per_run-%s-%s.csv" % (directory_expe, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
print ('Detailled data will be written in %s' % prep_file)
prep_each_run.to_csv(prep_file, mode='a')

print(prep_each_run)

## How to assess to the different values of the dataframe "prep_each_run"
#print(prep_each_run['Run1']['Well'][5])


## Analysis of bacteria growth using derivatives
### Analysis of bacteria growth in each wells according to a "while" loop

In [ ]:
# Number of the experiement (have to be automatised then)
exp = 1

# CSV file name to write detailed data
det_csvfile = "./%s/detailled-data_img-analysis_expe_%d-%s-%s.csv" % (directory_expe, exp, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
print ('Detailled data will be written in %s' % det_csvfile)

with open(det_csvfile, 'a') as det_csv:
        det_datawriter = csv.writer(det_csv)
        det_datawriter.writerow(['time', 'duration', 'well_x', 'well_y', 'crop size', 'iteration no.', 'sampling no.', 'Ave. grey scale'])

# CSV file name to write growth data
grth_csvfile = "./%s/growth-table-analysis_expe_%d-%s-%s.csv" % (directory_expe, exp, timenow.strftime("%Y%m%d"), timenow.strftime("%H%M%S"))
print ('The growth table will be written in %s' % grth_csvfile)

l_well = prep_each_run['Run1']['Well']
l_well.insert(0,'time')

with open(grth_csvfile, 'a') as gr_csv:
        gr_datawriter = csv.writer(gr_csv)
        gr_datawriter.writerow(l_well)  


        
# define the starting time of the experiment
timenow0 = datetime.now().replace(microsecond=0)


#Define variables for the loop 'while'
test_finished=0        # Condition to stop the loop
well_done=zeros(int(len(prep_each_run['Run1']['Well'])))    # List of the wells
k=0                    # Incremetation of runs
growth=zeros((2000,len(prep_each_run['Run1']['Well'])))  # Matrix for GrowthCurver (no need to add one for the time)
run=0                  # Starting value of runs

# Loop "while test_finished is different from 1 and run is bellow 10"
while (test_finished==0 and run < 10):
        
    # Start with plate shaking
    
    exp1.robot.move_to_firstxy([19.3*2.5+1.8, 19.3*4-1.8, 98])
    distance = 1.5
    for _ in range(10):
        exp1.robot.move_to([19.3*2.5+1.8-distance, 19.3*4-1.8, 98])
        exp1.robot.move_to([19.3*2.5+1.8+distance, 19.3*4-1.8, 98])
        exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8, 98])
        exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8+distance, 98])
        exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8-distance, 98])
        exp1.robot.move_to([19.3*2.5+1.8, 19.3*4-1.8, 98])
    exp1.robot.move_to_firstz([19.3*2.5+1.8, 19.3*4-1.8, init_z])
    
    # write the grey value with detailled of the well (det for detailled)
    with open(det_csvfile, 'a') as det_csv:
        det_datawriter = csv.writer(det_csv)
    
        # Calculate the time of experiement
        timenow = datetime.now().replace(microsecond=0)
        duration = timenow - timenow0
        duration_sde = duration.seconds

        # Loop for data acquisition
        growth[k,0]=duration_sde    ## Write the time of experiment in the first column

        for l in range (0,int(nb_row_plate)):         # Number of row (A, B, C, D)
            for c in range (0,int(nb_col_plate)):     # Number of column (1, 2, 3, 4, 5, 6)
                if(well_done[c+l*int(nb_col_plate)]==0):   # Condition to take te value (otherwise pass to the next well)

                    exp1.robot.move_to([init_y+c*well_pitch, init_x+l*well_pitch, init_z])
                    time.sleep(1) # wait 1s

                    # Ramp the camera - these frames will be discarded and are only used to allow v4l2
                    # to adjust light levels, if necessary
                    for foo in range(30):
                        temp = get_image()

                    # take a picture
                    for kk in range(0, 10):
                        img = get_image()
                        imgfile = "./%s/wastebot-%s-%.2d_%.2d-%.6d-%.2d-%s.png" % (directory_fullpic, timenow.strftime("%Y%m%d"), l, c, k, kk, timenow.strftime("%H%M%S"))
                        cv2.imwrite(imgfile, img)
                                                
                        # Tranform image into grey
                        crop_img_grey = cv2.cvtColor(img[cent_y-cropsize:cent_y+cropsize, cent_x-cropsize:cent_x+cropsize], cv2.COLOR_BGR2GRAY)
                        # Define the size of the circle
                        height = cropsize
                        width = cropsize
                        height, width = crop_img_grey.shape
                        circle_img = np.zeros((height, width), np.uint8)
                        cv2.circle(circle_img, (60, 60), 1, 1, thickness = 60)
                        # Paste the circle mask to the crop image
                        circle_grey_crop = cv2.bitwise_and (crop_img_grey, crop_img_grey, mask = circle_img)
                        
                        # Save the cropped picture in the croppic folder
                        outfile = "./%s/cropped_wastebot-%s-expe%s-%.2d_%.2d-%.6d-%.2d-%s-grey.png" % (directory_croppic, timenow.strftime("%Y%m%d"), exp, l, c, k, kk, timenow.strftime("%H%M%S"))
                        cv2.imwrite(outfile, circle_grey_crop)
                    
                    # Calculate the grey value and fill the detailled table
                    grey_value= np.average(circle_grey_crop)
                    print ("%s, %s, %d, %d, %d, %d, %d, %f" % (timenow, duration_sde, l, c, cropsize, k, kk, np.average(circle_grey_crop)))
                    det_datawriter.writerow([timenow, duration_sde, l, c, cropsize, k, kk, np.average(circle_grey_crop)])

                    # Fill the growthcurver table with the grey value
                    growth[k,1+(c+(l*int(nb_col_plate)))]=grey_value
                    
                    
                    # After 10 runs, calculate the derivative
                    if k > 15:  # change to 15 once of tests are done
                        
                        # Modelise the value to have a more precise derivative
                        spl = stats.smooth_spline(growth[0:(k-1),0], y=list(growth[0:(k-1),1+(c+(l*int(nb_col_plate)))]), spar=0.65)
                        pred3 = pd.DataFrame.from_dict(dict(zip((stats.predict(spl)).names, map(list,list(stats.predict(spl))))))
                        # Calculate the 1rst derivative and modelise it
                        deriv = (np.diff(pred3.y))/(np.diff(pred3.x))
                        spl_d1 = stats.smooth_spline(growth[1:(k-1),0], y=deriv, spar=0.65)
                        pred3_d1 = pd.DataFrame.from_dict(dict(zip((stats.predict(spl_d1)).names, map(list,list(stats.predict(spl_d1))))))
                        # Calculate the 2sd derivative and modelise it
                        deriv_2 = (np.diff(pred3_d1.y))/(np.diff(pred3_d1.x)) 
                        spl_d2 = stats.smooth_spline(growth[2:(k-1),0], y=deriv_2, spar=0.65)
                        pred3_d2 = pd.DataFrame.from_dict(dict(zip((stats.predict(spl_d2)).names, map(list,list(stats.predict(spl_d2))))))
                        
                        # condition to stop the analysis of the well
                        if sum(1 for number in (pred3_d2["y"].iloc[:-2]) if number < 0) > 0 and pred3_d2["y"].iloc[-1] > 0:
                            well_done[c+l*int(nb_col_plate)]=1
    
    # Save the growth table into a growth CSV file
    with open(grth_csvfile, 'a') as gr_csv:
        gr_datawriter = csv.writer(gr_csv)
        gr_datawriter.writerow(growth[k,])
    
    test_finished=1
    for well in range (0,24):
        if well_done[well]==0:
            test_finished=0 
    #print(growth)
        
    
    # increment k at the end of each loop 'while'
    k = k + 1
    
    time.sleep(3*60)
    

# Script to test growth parameters analysis

In [ ]:
# Analysis of growth parameter using the growth table
# Growth table stored in file: "grth_csvfile"

## Activate the line below if needed to change the directory
#os.chdir('Test_Auto_stop-20180316-091113')


greyval = pd.read_csv('growth-table-analysis_expe_1-20180316-110310.csv')      # read the csv file

print(greyval)

## Growth modelisation with logistic function
gr_param = gc.SummarizeGrowthByPlate(greyval, bg_correct = "min")

# Add the concentration corresponding to the wells
growth_param = pandas2ri.ri2py_dataframe(gr_param) ## conver R dataframe into pandas dataframe
#growth_param['Concentration']=prep_each_run['Run1']['Place of blanks'][0]   # activate in normal experiement
growth_param['Concentration']=['1.05','B','4.55','B','1.05','B',
                               'B','4.55','1.05','4.55','B','B',
                               '4.55','1.05','B','B','B','4.55',
                               'B','4.55','B', '1.05','B','1.05']


#print(growth_param)

## Create a button select the parameter to plot
columns = growth_param.columns.tolist()
selection = widgets.Dropdown(description="Parameter to plot",
                             options= list(growth_param.columns.tolist()))
display(selection)

## Create a button select the color of the plot
cols = widgets.Dropdown(description="Color:", options = ['maroon', 'darksalmon', 'sandybrown',
                                                         'goldenrod', 'olivedrab', 'limegreen',
                                                        'mediumseagreen', 'darkslategray','steelblue',
                                                         'mediumslateblue', 'indigo', 'mediumvioletred',
                                                         'deeppink', 'lightpink', 'k', 'w'])
display(cols)
                          
## Create button to display the graph
def on_button_clicked(b):
    sns.set_style("whitegrid")
    sns.set_context(rc={"figure.figsize": (12, 9)})
    sns.boxplot(x='Concentration', y=selection.value, data=growth_param, color=cols.value)
    sns.swarmplot(x="Concentration", y=selection.value, data=growth_param, color=".25")

button = widgets.Button(description="Make graph")
display(button)

button.on_click(on_button_clicked)

# Script to remove bubles from the analysis

In [ ]:
from PIL import Image   ## import the library
import os    ## import the directory
os.chdir("C:/Users/lc273k/Documents/Wastebot/Wastebot_Project/python/bubbles")    #change the directory
im = Image.open("cropped_wastebot-20180215-concept-00_00-000000-00-111146-grey.png")   # open the piacture
#im.show()       ## open the image in a new window

pixels = list(im.getdata())      # get a list of all the value of each pixel composing the image
#width, height = im.size       ## know the size of the image
new_pix = pixels

new_pix2 =[x if x <= 190 else 0 for x in new_pix]
##pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]  ## make a matrix of the value of all pixels


imNew=Image.new(im.mode ,im.size)   ## create a new image with the pixel of the previous list
imNew.putdata(new_pix2)    ## following code to create the new image
imNew.show()    ## open the new image ina new window

im3 = Image.open("cropped_wastebot-20180215-concept-03_04-000042-09-154258-grey.png")   ## example of TF image without bubbles
pixels3 = list(im3.getdata())
#im4 = Image.open("cropped_wastebot-20180215-concept-03_01-000042-09-154238-grey.png")   ## example of TF image with bubbles
#pixels4 = list(im4.getdata())

max(new_pix2)

## now, find a way to remove all pixel above 200?

In [ ]:
import PIL
from PIL import Image   
import glob
import os    ## import the directory
os.chdir("C:/Users/lc273k/Documents/Wastebot/Wastebot_Project/python/bubbles_run2")    #change the directory

no_bubble = 'no_bubble'
# create a directory if it doesn't exist
if not os.path.exists(no_bubble):
    os.makedirs(no_bubble)

outcsvfile_nobub = "./%s/wastebot-img_analysis_run3_no_bubble.csv" % (no_bubble)
print ('Output data below will be written in %s' % outcsvfile_nobub)

with open(outcsvfile_nobub, 'a') as mycsv:
    datawriter = csv.writer(mycsv)
    datawriter.writerow(['file', 'New ave. grey scale'])

    for filename in glob.glob("*.png"):
        im=Image.open(filename)
        pixels = list(im.getdata())
        new_pix =[x if x <= 190 else 0 for x in pixels]
        imNew=Image.new(im.mode ,im.size)   ## create a new image with the pixel of the previous list
        imNew.putdata(new_pix)    ## following code to create the new image
        #imNew.show()    ## open the new image in a new window
        imNew.save("./%s/new-%s.png" % (no_bubble, filename))
        print ("%s, %f" % (filename, np.average(new_pix)))
        datawriter.writerow([filename, np.average(new_pix)])